<div style="display: flex; gap: 10px;">
  <img src="../packages/cadfiles/HOOPS_AI.jpg" style="width: 20%;">
</div>

In [1]:
import hoops_ai
import os

hoops_ai.set_license(hoops_ai.use_test_license(), validate= False)

ℹ️ Using TEST LICENSE (expires December 8, 2025 - 40 days remaining)
   For production use, obtain your own license from Tech Soft 3D


In [2]:
import pathlib, sys
sys.path.insert(0, str( pathlib.Path.cwd()))
import importlib
import helper_tutorials  
importlib.reload(helper_tutorials)  # Forces reload of the latest version

<module 'helper_tutorials' from 'C:\\Users\\LuisSalazar\\Documents\\MAIN\\MLProject\\repo\\HOOPS-AI-tutorials\\notebooks\\helper_tutorials.py'>

# HOOPS AI: Using a Pre-trained Model

In this tutorial, we will use a pre-trained model to detect machine features. The model was trained using HOOPS AI, specifically using the pre-built architecture of GraphNodeClassification. The dataset contained more than 160,000 cad files and the splitting is as follows: 

Train set contains          : 113878 samples (70.12%)
Validation set contains     : 24220 samples (14.91%)
Test set contains           : 24311 samples (14.97%)
Total samples               : 162409

The model reported a IoU (Intersection of union) accuracy of 98.9% on the test set.

The dataset consists of synthetic samples describing the features (cadsynth dataset) and sample using features intersected (mfinst dataset). Both dataset are public.

<div style="display: flex; gap: 10px;">
  <img src="../packages/cadfiles/features_sampling_dataset.png" style="width: 50%;">
  <img src="../packages/cadfiles/features_intersection_dataset.png" style="width: 50%;">
</div>


Then, the model was tested on unseen data, real samples parts. The model was able to detect some features in real cases proving that besides simple trained data the model could learn how to detect the features.





In [3]:
from hoops_ai.dataset import DatasetExplorer

flow_name = "cadsynth_aag"
flow_root_dir = pathlib.Path.cwd().parent.joinpath("packages", "flows", flow_name)

parquet_file_path        = str(flow_root_dir.joinpath(f"{flow_name}.infoset"))
merged_store_path     = str(flow_root_dir.joinpath(f"{flow_name}.dataset"))
parquet_file_attribs  = str(flow_root_dir.joinpath(f"{flow_name}.attribset"))

explorer = DatasetExplorer(merged_store_path=merged_store_path, parquet_file_path=parquet_file_path, parquet_file_attribs=parquet_file_attribs)
#explorer.print_table_of_contents()

ValueError: Parquet file not found: C:\Users\LuisSalazar\Documents\MAIN\MLProject\repo\HOOPS-AI-tutorials\packages\flows\cadsynth_aag\cadsynth_aag.infoset

In [ ]:
face_dist = explorer.create_distribution(key="face_labels", bins=None, group="faces")
helper_tutorials.print_distribution_info(face_dist, title="Labels")

In [ ]:
dist = explorer.create_distribution(key="num_nodes", bins=12, group="graph")
helper_tutorials.print_distribution_info(dist, title="Brep facecount Distribution")

In [ ]:

import pathlib
import json

# Define paths 
demofile_dir  = pathlib.Path.cwd().parent.joinpath("packages", "cadfiles")
output_dir    = pathlib.Path.cwd().joinpath("out")
cad_file_test = str(demofile_dir.joinpath("nist_ftc_06_asme1_rd_sw1802.SLDPRT")) # largest_mftrcad.step
#cad_file_test = str(demofile_dir.joinpath("largest_mftrcad.step")) # 


In [ ]:
from hoops_ai.cadaccess import HOOPSLoader
from hoops_ai.insights import CADViewer

# Initialize CAD loader (needed for ML inference later)
loader = HOOPSLoader()

# Create viewer and load CAD file (auto-converts to SCS and displays in sidecar)
viewer = CADViewer(display_mode='sidecar', silent=True, static_folder=output_dir)
viewer.load_cad_file(cad_file_test, auto_convert=True, white_background=True)



In [ ]:
viewer.show()

# ML prediction 

The `FlowInference` class handles inference with trained models. It provides methods for preprocessing CAD data, making predictions, and post-processing results.

```python
class FlowInference:
    def __init__(self, cad_loader: CADLoader, flowmodel: FlowModel, log_file: str = 'training_errors.log'):
    
    def load_from_checkpoint(self, checkpoint_path: str):
        
    def preprocess(self, file_path: str) -> Dict[str, torch.Tensor]:

    def predict_and_postprocess(self, batch: Dict[str, torch.Tensor]) -> np.ndarray:

```

## Transform the CAD data into machine learning input data

- load the trained modle from the checkpoint file and instantiate using the respective FlowModel
- Initialize HoopsLoader to read the cad file
- preprocess the cad file to encode its data according to the expected ml architecture

In [ ]:
from hoops_ai.ml.EXPERIMENTAL import FlowInference
from hoops_ai.ml.EXPERIMENTAL import GraphNodeClassification

trained_model = pathlib.Path.cwd().parent.joinpath("packages","trained_ml_models", "ts3d_162k_mfr.ckpt")

inference_model = FlowInference(cad_loader = loader, flowmodel = GraphNodeClassification(result_dir=output_dir))
inference_model.load_from_checkpoint(trained_model)

We choose for the inference test a file from the NIST list. 

In [ ]:
ml_input = inference_model.preprocess(cad_file_test)    
predictions, probabilities = inference_model.predict_and_postprocess(ml_input)
print(predictions)

# Visualize Predictions on CAD Model

Now that we have predictions for each face, we'll use the **HOOPS AI Insights** module to visualize them on the 3D model.

## Step 1: Create a Color Palette

We use `ColorPalette` to map each manufacturing feature label to a unique color and description. We can optionally reserve specific colors for important features (e.g., red for through holes).


In [ ]:
from hoops_ai.insights.utils import ColorPalette

# Define label descriptions
labels_description = {
    0: "no-label", 1: "rectangular_through_slot", 2: "triangular_through_slot", 3: "rectangular_passage",
    4: "triangular_passage", 5: "6sides_passage", 6: "rectangular_through_step", 7: "2sides_through_step",
    8: "slanted_through_step", 9: "rectangular_blind_step", 10: "triangular_blind_step", 
    11: "rectangular_blind_slot", 12: "rectangular_pocket", 13: "triangular_pocket", 14: "6sides_pocket",
    15: "chamfer", 16: "circular through slot", 17: "through hole", 18: "circular blind step",
    19: "horizontal circular end blind slot", 20: "vertical circular end blind slot", 
    21: "circular end pocket", 22: "o-ring", 23: "blind hole", 24: "fillet"
}

# Create color palette with custom colors for specific features
color_palette = ColorPalette.from_labels(
    labels_description,
    cmap_name='hsv',
    reserved_colors={
        0: (200, 200, 200),  # Gray for background
    }
)

print(f"Created color palette for {len(color_palette)} labels")
print(f"Sample: Label 17 -> Color {color_palette.get_color(17)}, Description: '{color_palette.get_description(17)}'")
#help(ColorPalette)

### Understanding the Workflow

The visualization workflow has two main steps:

**1. Group predictions by label:**
```python
face_groups = group_predictions_by_label(predictions, color_palette, exclude_labels={0})
```
This returns a list of `(face_indices, color, description)` tuples, one for each unique label.

**2. Color the faces:**
```python
viewer.color_faces_by_groups(face_groups, delay=0.5, verbose=True)
```
This colors all face groups with animated progress feedback.

**Manual approach (if you need full control):**
```python
viewer.clear_face_colors()
for face_indices, color, description in face_groups:
    viewer.set_face_color(face_indices, list(color))
    print(f"{description}: {len(face_indices)} faces")
```


## Step 2: Group Predictions and Visualize

Now we visualize the predictions using three simple lines:
1. **Group** faces by their predicted label using `group_predictions_by_label()`
2. **Visualize** on the viewer using `color_faces_by_groups()`
3. Enjoy **animated coloring** with progress feedback!


In [ ]:
import hoops_ai

# Group faces by prediction label and attach colors/descriptions
face_groups = hoops_ai.insights.utils.group_predictions_by_label(predictions, color_palette, exclude_labels={0})

# Visualize on the viewer
viewer.color_faces_by_groups(face_groups, delay=0.5, verbose=True)


In the Hoops viewer, you can see how the features detected are projected in the cad file

# A second case

In [ ]:
# Load second CAD file
cad_file_test = str(demofile_dir.joinpath("largest_mftrcad.step"))

# Load new model in existing viewer (auto-converts to SCS)
viewer.load_cad_file(cad_file_test, auto_convert=True, white_background=True)


In [ ]:
# Run inference on second model
ml_input = inference_model.preprocess(cad_file_test)    
predictions, probabilities = inference_model.predict_and_postprocess(ml_input)
print(f"Predictions shape: {predictions.shape}")


In [ ]:

# Group and visualize (reusing color_palette from above)
face_groups_2 = hoops_ai.insights.utils.group_predictions_by_label(predictions, color_palette, exclude_labels={0})
viewer.color_faces_by_groups(face_groups_2, delay=0.5, verbose=True)


In [ ]:
viewer.terminate()